Notre preprocessing comporte : la conversion de séquence d'ADN (string) en représentation 2D et l'ajout d'un padding pour remédier à la taille variable des séquences.

In [ ]:
#@title Fonctions conversions

import numpy as np

# Convertir les séquences en image
def convert_to_pairings(seq):

    # seq = seq[0]
    result = []

    AT_pair = []
    TA_pair = []
    CG_pair = []
    GC_pair = []
    unpaired = []

    for i in range(len(seq)):
        # base pairings
        AT_pair_row = [0] * len(seq)
        TA_pair_row = [0] * len(seq)
        CG_pair_row = [0] * len(seq)
        GC_pair_row = [0] * len(seq)
        unpaired_row = [1] * len(seq)

        for j in range(len(seq)):
            if (seq[i] == 'A' and seq[j] == 'T'):
                AT_pair_row[j] = 1
                unpaired_row[j] = 1
            elif (seq[i] == 'T' and seq[j] == 'A'):
                TA_pair_row[j] = 1
                unpaired_row[j] = 1
            elif (seq[i] == 'C' and seq[j] == 'G'):
                CG_pair_row[j] = 1
                unpaired_row[j] = 1
            elif (seq[i] == 'G' and seq[j] == 'C'):
                GC_pair_row[j] = 1
                unpaired_row[j] = 0

        AT_pair.append(AT_pair_row)
        TA_pair.append(TA_pair_row)
        CG_pair.append(CG_pair_row)
        GC_pair.append(GC_pair_row)
        unpaired.append(unpaired_row)

    result.append(AT_pair)
    result.append(TA_pair)
    result.append(CG_pair)
    result.append(GC_pair)
    result.append(unpaired)

    return np.array(result)

# Appliquer cette conversion à tout un dataset
def convert_to_pairings_dataset(dataset):
    result = []
    for seq in dataset:
        result.append(convert_to_pairings(seq))
    return result

# Convertir les strings en image 2D
def structure_to_matrix(structure):
    length = len(structure)
    matrix = np.zeros((length, length))
    stack = []

    for i, symbol in enumerate(structure):
        if symbol == '(':
            stack.append(i)
        elif symbol == ')':
            j = stack.pop()
            matrix[i][j] = 1
            matrix[j][i] = 1

    return matrix

# Ajoute un padding car séquences de taille variable
def pad_structure(matrix, common_size, num_channels=None):
    if num_channels:
        padded_matrix = np.zeros((num_channels, *common_size))
        for i in range(num_channels):
            padded_matrix[i, :matrix.shape[1], :matrix.shape[2]] = matrix[i]
    else:
        padded_matrix = np.zeros(common_size)
        padded_matrix[:matrix.shape[0], :matrix.shape[1]] = matrix
    return padded_matrix



La conversion des séquences en image + padding est couteux pour la mémoire vive, il faut donc répartir nos images en batch pour qu'elles soient chargées/déchargées en mémoire au besoin, sans surcharger notre RAM.

In [ ]:
#@title Preprocessing - Batching the data

import pandas as pd
import numpy as np
import os

import torch
from torch.utils.data import DataLoader

# Load données
def load_data(file_path):
    return pd.read_pickle(file_path)

# Conversion des données + padding
def preprocess_and_convert(batch, common_size=(60, 60)):

    # Convert sequences to pairings
    # dataset_X = [convert_to_pairings(seq[0]) for seq in batch]
    # dataset_Y = [seq[1] for seq in batch]  # Assuming second element is the label
    # structures = [structure_to_matrix(seq[2]) for seq in batch]  # Assuming third element is the structure

    dataset_X = [convert_to_pairings(seq) for seq in batch[0]]
    dataset_Y = [seq for seq in batch[1]]  # Assuming second element is the label
    structures = [structure_to_matrix(seq) for seq in batch[2]]  # Assuming third element is the structure
    hairpins = [seq for seq in batch[3]]  # Assuming third element is the hairpins

    # Pad sequences and structures
    X_padded = [pad_structure(x, common_size, num_channels=5) for x in dataset_X]
    # Y_padded = [pad_structure(y, common_size) for y in structures]  # Pad structures if they are labels

    # return X_padded, Y_padded  # Return padded features and labels
    # return X_padded, dataset_Y  # Return padded features and labels
    return X_padded, hairpins  # Return padded features and labels


# Function to save features and labels to a file
def save_batch(features, labels, batch_index, output_dir):
    batch_file = os.path.join(output_dir, f'batch_{batch_index}.npz')
    np.savez_compressed(batch_file, features=features, labels=labels)

# Function to process, preprocess (pad), and save the entire dataset in batches
def process_and_save_batches(file_path, batch_size=100000, output_dir='batches_hairpins', common_size=(60, 60)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    data = load_data(file_path)

    if isinstance(data, torch.utils.data.dataset.Subset):

        data_loader = DataLoader(data, batch_size=batch_size, shuffle=False)
        total_batches = len(data_loader)

        for i, batch_data in enumerate(data_loader):

            print(f'Starting Batch {i+1}/{total_batches}.')

            X_padded, Y_padded = preprocess_and_convert(batch_data, common_size)
            save_batch(X_padded, Y_padded, i, output_dir)
            print(f'Batch {i+1}/{total_batches} processed and saved.')

    elif isinstance(data, list):
        total_batches = (len(data) // batch_size) + (1 if len(data) % batch_size != 0 else 0)
        for i in range(total_batches):
            start_index = i * batch_size
            end_index = start_index + batch_size
            batch_data = data[start_index:end_index]

            X_padded, Y_padded = preprocess_and_convert(batch_data, common_size)
            save_batch(X_padded, Y_padded, i, output_dir)
            print(f'Batch {i+1}/{total_batches} processed and saved.')

    else:
        raise TypeError("Unsupported data type for batching")


file_path = '/content/gdrive/My Drive/train_5M_struct.pkl'
batch_size = 100000
process_and_save_batches(file_path, batch_size)


Les architectures des modèles utilisés sont différentes dépendamment du type de donnée à prédire. Dans le cas des MFE (Float) et hairpins (Int), on utilise SimpleCNN(). Pour les structures secondaires (Matrice 2D) on utilise StructureCNN().

In [ ]:
#@title Architecture CNN

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(5, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        # self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        # self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        # self.bn5 = nn.BatchNorm2d(512)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Dropout layer
        self.dropout = nn.Dropout(0.2)

        self.fc1 = nn.Linear(128 * 15 * 15, 512)
        # self.fc1 = nn.Linear(512 * 15 * 15, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        # x = F.relu(self.conv1(x))
        # x = self.pool(x)
        # x = F.relu(self.conv2(x))
        # x = F.relu(self.conv3(x))
        # x = self.adaptive_pool(x)  # Adaptive pooling layer
        # x = x.view(x.size(0), -1)
        # x = F.relu(self.fc1(x))
        # x = self.fc2(x)

        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        # x = F.relu(self.bn4(self.conv4(x)))
        # x = F.relu(self.bn5(self.conv5(x)))

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

# class StructureCNN(nn.Module):
#     def __init__(self, num_channels, output_size):
#         super(StructureCNN, self).__init__()
#         self.output_size = output_size
#         self.conv1 = nn.Conv2d(num_channels, 16, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.adaptive_pool = nn.AdaptiveAvgPool2d((self.output_size, self.output_size))
#         self.fc1 = nn.Linear(64 * self.output_size * self.output_size, 512)
#         self.fc2 = nn.Linear(512, self.output_size * self.output_size)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = self.adaptive_pool(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         x = x.view(-1, self.output_size, self.output_size)
#         x = torch.sigmoid(x)
#         return x

class StructureCNN(nn.Module):
    def __init__(self, num_channels, output_size):
        super(StructureCNN, self).__init__()
        self.output_size = output_size

        # Convolutional layers with batch normalization
        self.conv1 = nn.Conv2d(num_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        # self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        # self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        # self.bn5 = nn.BatchNorm2d(512)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Dropout layer
        self.dropout = nn.Dropout(0.5)

        # Fully connected layers
        reduced_size = output_size // 2**2  # Adjust based on the number of pooling layers
        self.fc1 = nn.Linear(512 * reduced_size * reduced_size, 1024)
        self.fc2 = nn.Linear(1024, output_size * output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        # x = F.relu(self.bn4(self.conv4(x)))
        # x = F.relu(self.bn5(self.conv5(x)))

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        # Reshape to the output dimension and apply sigmoid activation
        x = x.view(-1, self.output_size, self.output_size)
        x = torch.sigmoid(x)
        return x

Lors du training, on load les batches d'images nécessaires en mémoire.L'entrainement du model est lent, il nous fallait donc plusieurs sessions de training. Pour la prédiction des MFE (float) et hairpins (int) nous utilisons MSE (Mean Squared Error) comme loss. Pour la prédiction des structures secondaires, nous utilisons BCE (Binary Cross Entropy).

In [ ]:
#@title Train model

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os
import gc
import torch
from torch.utils.data import DataLoader, TensorDataset
import os

# Function to load a batch from file, which includes features and labels
def load_batch(file_path):
    with np.load(file_path) as data:
        return data['features'], data['labels']

# Function to train the model using preprocessed batches from a directory
def train_model_from_batches(batch_dir, model, criterion, optimizer, num_epochs=100, batch_size=128):
    batch_files = [os.path.join(batch_dir, f) for f in os.listdir(batch_dir) if f.endswith('.npz')]
    print(batch_files)


    for epoch in range(num_epochs):
        print(f'Starting epoch {epoch+1}/{num_epochs}')

        sum_loss = 0
        total_size = 0
        avg_val_loss = 0

        for batch_file in batch_files:

            print(f'Loading {batch_file}')
            features, labels = load_batch(batch_file)
            batch_loss = 0

            # Convert numpy arrays to tensors
            features_tensor = torch.tensor(features, dtype=torch.float32)
            labels_tensor = torch.tensor(labels, dtype=torch.float32)

            # Split the batch into training and validation sets
            X_train, X_val, y_train, y_val = train_test_split(features_tensor, labels_tensor, test_size=0.1, random_state=42)

            # Create data loaders
            train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)


            # Training loop
            model.train()
            for inputs, labels in train_loader:

                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                # if predicting mfe
                outputs = outputs.squeeze()

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                sum_loss += loss.item()
                batch_loss += loss.item()

            total_size += len(train_loader)

            # Validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = model(inputs)
                    outputs = outputs.squeeze()

                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                val_loss /= len(val_loader)

            print(f'Epoch {epoch+1}, Batch processed, Batch Loss :{batch_loss / len(train_loader)} , Validation Loss: {val_loss:.6f}')
            avg_val_loss += val_loss

            # Free memory after processing each batch
            del features, labels, features_tensor, labels_tensor, X_train, X_val, y_train, y_val, train_loader, val_loader
            torch.cuda.empty_cache()  # If using CUDA

        average_loss = sum_loss / total_size
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.6f}, Average val loss: {avg_val_loss / len(batch_files):.6f}', end='')

        # torch.save(model.state_dict(), 'model_mfe.pth')
        # !cp model_mfe.pth '/content/gdrive/My Drive/'



batch_dir = 'batches_hairpins'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# model = StructureCNN(num_channels=5, output_size=60).to(device)
model = SimpleCNN().to(device)

# Continue the training
# model.load_state_dict(torch.load('/content/gdrive/My Drive/model.pth'))
# model.load_state_dict(torch.load('/content/gdrive/My Drive/model_mfe.pth'))
# model.load_state_dict(torch.load('/content/gdrive/My Drive/model_hairpins.pth'))

# criterion = torch.nn.BCELoss()
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)

train_model_from_batches(batch_dir, model, criterion, optimizer)

Une fois le modèle entrainé, on le test sur l'ensemble de test. Pour le rapport, il était important de regarder la performance du modèle sur des tailles de séquences différentes. On sépare donc les données en fonction de la longueur de la séquence. On applique ensuite le preprocessing sur ces données (Séquence vers image + padding et structure vers image + padding).

In [ ]:
#@title Load and Preprocess test data

import pandas as pd

data = pd.read_pickle('data_test.pkl')

#Garder seulement les séquences de longueur <= 50
data = [seq for seq in data if len(seq[0]) <= 50]

#Séparer les séquences par longueur (de 10 a 20)
data_10_20 = [seq for seq in data if len(seq[0]) <= 20]
data_20_30 = [seq for seq in data if len(seq[0]) <= 30 and len(seq[0]) > 20]
data_30_40 = [seq for seq in data if len(seq[0]) <= 40 and len(seq[0]) > 30]
data_40_50 = [seq for seq in data if len(seq[0]) <= 50 and len(seq[0]) > 40]

# Features
data_X = [convert_to_pairings(seq[0]) for seq in data]
data_X = [pad_structure(matrix, (60, 60), 5) for matrix in data_X]

data_10_20_X = [convert_to_pairings(seq[0]) for seq in data_10_20]
data_10_20_X = [pad_structure(matrix, (60, 60), 5) for matrix in data_10_20_X]

data_20_30_X = [convert_to_pairings(seq[0]) for seq in data_20_30]
data_20_30_X = [pad_structure(matrix, (60, 60), 5) for matrix in data_20_30_X]

data_30_40_X = [convert_to_pairings(seq[0]) for seq in data_30_40]
data_30_40_X = [pad_structure(matrix, (60, 60), 5) for matrix in data_30_40_X]

data_40_50_X = [convert_to_pairings(seq[0]) for seq in data_40_50]
data_40_50_X = [pad_structure(matrix, (60, 60), 5) for matrix in data_40_50_X]

# Labels
data_Y_mfe = [seq[1] for seq in data]
data_Y_hairpins = [seq[3] for seq in data]

data_Y_struct = [structure_to_matrix(seq[2]) for seq in data]
data_Y_struct = [pad_structure(matrix, (60, 60)) for matrix in data_Y_struct]

data_10_20_Y_mfe = [seq[1] for seq in data_10_20]
data_10_20_Y_hairpins = [seq[3] for seq in data_10_20]

data_10_20_Y_struct = [structure_to_matrix(seq[2]) for seq in data_10_20]
data_10_20_Y_struct = [pad_structure(matrix, (60, 60)) for matrix in data_10_20_Y_struct]

data_20_30_Y_mfe = [seq[1] for seq in data_20_30]
data_20_30_Y_hairpins = [seq[3] for seq in data_20_30]

data_20_30_Y_struct = [structure_to_matrix(seq[2]) for seq in data_20_30]
data_20_30_Y_struct = [pad_structure(matrix, (60, 60)) for matrix in data_20_30_Y_struct]

data_30_40_Y_mfe = [seq[1] for seq in data_30_40]
data_30_40_Y_hairpins = [seq[3] for seq in data_30_40]

data_30_40_Y_struct = [structure_to_matrix(seq[2]) for seq in data_30_40]
data_30_40_Y_struct = [pad_structure(matrix, (60, 60)) for matrix in data_30_40_Y_struct]

data_40_50_Y_mfe = [seq[1] for seq in data_40_50]
data_40_50_Y_hairpins = [seq[3] for seq in data_40_50]

data_40_50_Y_struct = [structure_to_matrix(seq[2]) for seq in data_40_50]
data_40_50_Y_struct = [pad_structure(matrix, (60, 60)) for matrix in data_40_50_Y_struct]

On calcule ensuite les prédictions du modèle sur nos ensembles de tests séparés.

In [ ]:
#@title Get model prediction

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, recall_score, f1_score

def matrix_to_structure(matrix):
    structure = ''
    for i in range(matrix.shape[0]):
        if sum(matrix[i]) == 0:
            structure += '.'
        else:
            for j in range(matrix.shape[1]):
                if matrix[i, j] == 1:
                    if i < j:
                        structure += '('
                    else:
                        structure += ')'
    return structure

def evaluate_model(model, features,labels, batch_size=128, isStruct = False):

    features_tensor = torch.tensor(features, dtype=torch.float32).to(device)
    labels_tensor = torch.tensor(labels, dtype=torch.float32).to(device)

    test_loader = DataLoader(TensorDataset(features_tensor, labels_tensor), batch_size=batch_size, shuffle=False)

    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)

            # Forward pass and round the outputs
            outputs = model(inputs).cpu().numpy()

            if(isStruct):
              outputs_rounded = np.round(outputs)

              for output in outputs_rounded:
                  predictions.append(matrix_to_structure(output))

              for label in labels.cpu().numpy():
                  actuals.append(matrix_to_structure(label))


            else:
              for output in outputs.squeeze():
                  predictions.append(output)

              for label in labels.cpu().numpy():
                  actuals.append(label)

    return predictions, actuals

# Call the function to evaluate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Si on test les structures secondaires

# model = StructureCNN(num_channels=5, output_size=60).to(device)

# model.load_state_dict(torch.load('/content/gdrive/My Drive/model.pth'))

# pred_data,actual_data = evaluate_model(model, features = data_X, labels = data_Y_struct)
# pred_data_10_20,actual_data_10_20 = evaluate_model(model, features = data_10_20_X, labels = data_10_20_Y_struct)
# pred_data_20_30,actual_data_20_30 = evaluate_model(model, features = data_20_30_X, labels = data_20_30_Y_struct)
# pred_data_30_40,actual_data_30_40 = evaluate_model(model, features = data_30_40_X, labels = data_30_40_Y_struct)
# pred_data_40_50,actual_data_40_50 = evaluate_model(model, features = data_40_50_X, labels = data_40_50_Y_struct)

# # Si on test MFE

# model = SimpleCNN().to(device)

# model.load_state_dict(torch.load('/content/gdrive/My Drive/model_mfe.pth'))

# pred_data,actual_data = evaluate_model(model, features = data_X, labels = data_Y_mfe)
# pred_data_10_20,actual_data_10_20 = evaluate_model(model, features = data_10_20_X, labels = data_10_20_Y_mfe)
# pred_data_20_30,actual_data_20_30 = evaluate_model(model, features = data_20_30_X, labels = data_20_30_Y_mfe)
# pred_data_30_40,actual_data_30_40 = evaluate_model(model, features = data_30_40_X, labels = data_30_40_Y_mfe)
# pred_data_40_50,actual_data_40_50 = evaluate_model(model, features = data_40_50_X, labels = data_40_50_Y_mfe)

# Si on test les hairpins

model = SimpleCNN().to(device)

model.load_state_dict(torch.load('/content/gdrive/My Drive/model_hairpins.pth'))

pred_data,actual_data = evaluate_model(model, features = data_X, labels = data_Y_hairpins)
pred_data_10_20,actual_data_10_20 = evaluate_model(model, features = data_10_20_X, labels = data_10_20_Y_hairpins)
pred_data_20_30,actual_data_20_30 = evaluate_model(model, features = data_20_30_X, labels = data_20_30_Y_hairpins)
pred_data_30_40,actual_data_30_40 = evaluate_model(model, features = data_30_40_X, labels = data_30_40_Y_hairpins)
pred_data_40_50,actual_data_40_50 = evaluate_model(model, features = data_40_50_X, labels = data_40_50_Y_hairpins)

Using device: cuda


Pour comparer les performances du modèle, nous calculons plusieurs métriques :

Mean Squared Error, Mean Average Error, Root Mean Squared Error, Le score R2 et la précision (accuracy) du modèle par rapport aux vraies valeurs.

R2 est calculé seulement pour MFE (car floats) et accuracy seulement pour hairpins et structures secondaires.

In [ ]:
#@title Get metrics

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calcul MSE / MAE
def calculate_metrics(predictions, actuals):
    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae

# Si on prédit hairpins (int) : arrondir prediction au int le plus proche

pred_data = np.round(pred_data)
pred_data_10_20 = np.round(pred_data_10_20)
pred_data_20_30 = np.round(pred_data_20_30)
pred_data_30_40 = np.round(pred_data_30_40)
pred_data_40_50 = np.round(pred_data_40_50)

# Calcule MSE / MAE pour chaque ensemble
mse_data, mae_data = calculate_metrics(pred_data, actual_data)
mse_data_10_20, mae_data_10_20 = calculate_metrics(pred_data_10_20, actual_data_10_20)
mse_data_20_30, mae_data_20_30 = calculate_metrics(pred_data_20_30, actual_data_20_30)
mse_data_30_40, mae_data_30_40 = calculate_metrics(pred_data_30_40, actual_data_30_40)
mse_data_40_50, mae_data_40_50 = calculate_metrics(pred_data_40_50, actual_data_40_50)

# Print MSE / MAE
print("MSE for data:", mse_data)
print("MAE for data:", mae_data)
print("MSE for data_10_20:", mse_data_10_20)
print("MAE for data_10_20:", mae_data_10_20)
print("MSE for data_20_30:", mse_data_20_30)
print("MAE for data_20_30:", mae_data_20_30)
print("MSE for data_30_40:", mse_data_30_40)
print("MAE for data_30_40:", mae_data_30_40)
print("MSE for data_40_50:", mse_data_40_50)
print("MAE for data_40_50:", mae_data_40_50)

# Define function to calculate RMSE
def calculate_rmse(predictions, actuals):
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    return rmse

# Calculate RMSE for each dataset
rmse_data = calculate_rmse(pred_data, actual_data)
rmse_data_10_20 = calculate_rmse(pred_data_10_20, actual_data_10_20)
rmse_data_20_30 = calculate_rmse(pred_data_20_30, actual_data_20_30)
rmse_data_30_40 = calculate_rmse(pred_data_30_40, actual_data_30_40)
rmse_data_40_50 = calculate_rmse(pred_data_40_50, actual_data_40_50)

# Print the RMSE for each dataset
print("RMSE for data:", rmse_data)
print("RMSE for data_10_20:", rmse_data_10_20)
print("RMSE for data_20_30:", rmse_data_20_30)
print("RMSE for data_30_40:", rmse_data_30_40)
print("RMSE for data_40_50:", rmse_data_40_50)

# Calcule R2
r2_data = r2_score(actual_data, pred_data)
r2_data_10_20 = r2_score(actual_data_10_20, pred_data_10_20)
r2_data_20_30 = r2_score(actual_data_20_30, pred_data_20_30)
r2_data_30_40 = r2_score(actual_data_30_40, pred_data_30_40)
r2_data_40_50 = r2_score(actual_data_40_50, pred_data_40_50)

# Print score R2
print("R2 for data:", r2_data)
print("R2 for data_10_20:", r2_data_10_20)
print("R2 for data_20_30:", r2_data_20_30)
print("R2 for data_30_40:", r2_data_30_40)
print("R2 for data_40_50:", r2_data_40_50)

# Print la précision (si hairpins ou structure secondaire)
print(accuracy_score(pred_data,actual_data))
print(accuracy_score(pred_data_10_20,actual_data_10_20))
print(accuracy_score(pred_data_20_30,actual_data_20_30))
print(accuracy_score(pred_data_30_40,actual_data_30_40))
print(accuracy_score(pred_data_40_50,actual_data_40_50))


MSE for data: 0.15960975
MAE for data: 0.1529756
MSE for data_10_20: 0.014545455
MAE for data_10_20: 0.014545455
MSE for data_20_30: 0.1004
MAE for data_20_30: 0.1
MSE for data_30_40: 0.212
MAE for data_30_40: 0.2044
MSE for data_40_50: 0.326
MAE for data_40_50: 0.3068
MAE percentage for data: 5.099186673760414
MAE percentage for data_10_20: 1.4545454643666744
MAE percentage for data_20_30: 5.000000074505806
MAE percentage for data_30_40: 6.813333183526993
MAE percentage for data_40_50: 10.226666927337646
MAE percentage relative to average for data: -11.527716368436813
MAE percentage relative to average for data_10_20: -1.4311270788311958
MAE percentage relative to average for data_20_30: -8.716876059770584
MAE percentage relative to average for data_30_40: -14.465676248073578
MAE percentage relative to average for data_40_50: -17.406105995178223
RMSE for data: 0.3995119
RMSE for data_10_20: 0.12060454
RMSE for data_20_30: 0.3168596
RMSE for data_30_40: 0.4604346
RMSE for data_40_50: 0

Si on calcule R2, on peut faire un graphe pour avoir une représentation des prédictions de notre modèle.

In [ ]:
#@title Plot R2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Calculate R2 score
r2 = r2_score(actual_data, pred_data)

# Plot the actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(actual_data, pred_data, color='blue', label='Actual vs Predicted')
plt.plot([min(actual_data), max(actual_data)], [min(actual_data), max(actual_data)], linestyle='--', color='red', label='Perfect Prediction')
plt.xlabel('Vraies valeurs')
plt.ylabel('Predictions')
plt.title(f'Predictions vs Labels (R2={r2:.2f})')
plt.legend()
plt.grid(True)
plt.show()


Si on prédit la structure secondaire, il est pertinent d'avoir plus de métriques de performance, telle que le score F1

In [ ]:
#@title More metrics if Secondary Structures
from sklearn.metrics import accuracy_score,precision_score

def compute_metrics(pred_data, actual_data):
    # Convert concatenated sequences back to lists of characters
    pred_data = list(pred_data)
    actual_data = list(actual_data)

    precision = precision_score(actual_data, pred_data, average=None, labels=[".", "(", ")"])
    recall = recall_score(actual_data, pred_data, average=None, labels=[".", "(", ")"])
    f1 = f1_score(actual_data, pred_data, average=None, labels=[".", "(", ")"])
    return precision, recall, f1

# Compute metrics for each fixed dataset and print the results
print("Metrics for pred_data:")
precision, recall, f1 = compute_metrics(''.join(pred_data), ''.join(actual_data))
print("Symbol '.', Precision:", precision[0], "Recall:", recall[0], "F1 Score:", f1[0])
print("Symbol '(', Precision:", precision[1], "Recall:", recall[1], "F1 Score:", f1[1])
print("Symbol ')', Precision:", precision[2], "Recall:", recall[2], "F1 Score:", f1[2])

# Repeat for other datasets
print("\nMetrics for pred_data_10_20:")
precision, recall, f1 = compute_metrics(''.join(pred_data_10_20), ''.join(actual_data_10_20))
print("Symbol '.', Precision:", precision[0], "Recall:", recall[0], "F1 Score:", f1[0])
print("Symbol '(', Precision:", precision[1], "Recall:", recall[1], "F1 Score:", f1[1])
print("Symbol ')', Precision:", precision[2], "Recall:", recall[2], "F1 Score:", f1[2])

print("\nMetrics for pred_data_20_30:")
precision, recall, f1 = compute_metrics(''.join(pred_data_20_30), ''.join(actual_data_20_30))
print("Symbol '.', Precision:", precision[0], "Recall:", recall[0], "F1 Score:", f1[0])
print("Symbol '(', Precision:", precision[1], "Recall:", recall[1], "F1 Score:", f1[1])
print("Symbol ')', Precision:", precision[2], "Recall:", recall[2], "F1 Score:", f1[2])

print("\nMetrics for pred_data_30_40:")
precision, recall, f1 = compute_metrics(''.join(pred_data_30_40), ''.join(actual_data_30_40))
print("Symbol '.', Precision:", precision[0], "Recall:", recall[0], "F1 Score:", f1[0])
print("Symbol '(', Precision:", precision[1], "Recall:", recall[1], "F1 Score:", f1[1])
print("Symbol ')', Precision:", precision[2], "Recall:", recall[2], "F1 Score:", f1[2])

print("\nMetrics for pred_data_40_50:")
precision, recall, f1 = compute_metrics(''.join(pred_data_40_50), ''.join(actual_data_40_50))
print("Symbol '.', Precision:", precision[0], "Recall:", recall[0], "F1 Score:", f1[0])
print("Symbol '(', Precision:", precision[1], "Recall:", recall[1], "F1 Score:", f1[1])
print("Symbol ')', Precision:", precision[2], "Recall:", recall[2], "F1 Score:", f1[2])



Accuracy for pred_data: 0.5595121951219513
Accuracy for pred_data_10_20: 0.8921818181818182
Accuracy for pred_data_20_30: 0.6568
Accuracy for pred_data_30_40: 0.438
Accuracy for pred_data_40_50: 0.2178
Metrics for pred_data:
Symbol '.', Precision: 0.9587253950206435 Recall: 0.9824694679389165 F1 Score: 0.9704522165378114
Symbol '(', Precision: 0.8840578015759815 Recall: 0.7690125088986067 F1 Score: 0.8225318713831962
Symbol ')', Precision: 0.875219211073959 Recall: 0.7613241126817858 F1 Score: 0.814308401862246

Metrics for pred_data_10_20:
Symbol '.', Precision: 0.9961555601375485 Recall: 0.9961356834053874 F1 Score: 0.9961456216723146
Symbol '(', Precision: 0.9613048522486862 Recall: 0.9615017064846416 F1 Score: 0.9614032692898338
Symbol ')', Precision: 0.9561864464614754 Recall: 0.956382252559727 F1 Score: 0.9562843394874245

Metrics for pred_data_20_30:
Symbol '.', Precision: 0.9789157770770809 Recall: 0.9857915636931223 F1 Score: 0.9823416389642868
Symbol '(', Precision: 0.8935001